In [ ]:
import time
import typing

import numpy as np
import pandas as pd
import torch
import scipy

import os
import json

In [ ]:
from google.colab import drive

In [ ]:
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
file_dir = '/content/gdrive/My Drive/msci720_prj/dataset/train-dataset/ml-implicit/training_set_user11k'
file = 'ratings.csv'
file_path = os.path.join(file_dir, file)

columns = ["userId", "movieId", "rating", "timestamp"]
ratings_train = pd.read_csv(file_path, sep=',')

In [ ]:
file_dir = '/content/gdrive/My Drive/msci720_prj/'
file = 'userIds.json'
file_path = os.path.join(file_dir, file)

In [ ]:
with open(file_path, "r") as json_file:
    userIds = json.load(json_file)

In [ ]:
# user_train_trimmed = pd.DataFrame({'userId': userIds})

In [ ]:
# training_set = pd.merge(ratings_train, user_train_trimmed, on=["userId"], how="inner")

In [ ]:
training_set = ratings_train
training_set

,userId,movieId,rating,timestamp
0,5,10,4.0,840768638
1,5,110,4.0,840768763
2,5,161,4.0,840764183
3,5,165,4.0,840764017
4,5,349,4.0,840764017
...,...,...,...,...
862942,200959,176371,4.5,1663748485
862943,200959,192365,5.0,1663748485
862944,200959,194474,5.0,1663748485
862945,200959,204698,4.0,1663748485


In [ ]:
user_item_binary_matrix = pd.crosstab(training_set["userId"], training_set["movieId"]).clip(upper=1)
user_item_binary_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,288979,289253,289727,289797,290213,290263,290383,290767,291419,291857
userId,,,,,,,,,,,,,,,,,,,,,
5,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
36,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200955,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
200956,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
200957,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
user_item = torch.FloatTensor(user_item_binary_matrix.values)
user_item

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
# train_dataloader = torch.utils.data.dataloader.DataLoader(user_item, batch_size=128)

In [ ]:
'''
for batch in train_dataloader:
    # Get the size of the data in the current batch
    batch_size = batch.size(0)  # Assuming the first element in each batch contains the data
    print("Batch size:", batch_size)
'''

'\nfor batch in train_dataloader:\n    # Get the size of the data in the current batch\n    batch_size = batch.size(0)  # Assuming the first element in each batch contains the data\n    print("Batch size:", batch_size)\n'

In [44]:
class Autoencoder(torch.nn.Module):
  def __init__(self, dim_item, lr=0.15, weight_decay=1e-3):
    super(Autoencoder, self).__init__()
    # W = torch.nn.Parameter(torch.nn.init.xavier_uniform_(torch.empty([dim_item, dim_item])).detach().clone())
    # self.W_list = torch.nn.ParameterList([W])
    self.encoder = torch.nn.Sequential(
      torch.nn.Linear(dim_item, 128),
      torch.nn.ReLU(),
      torch.nn.Linear(128, 64)
    )
    self.decoder = torch.nn.Sequential(
      torch.nn.Linear(64, 128),
      torch.nn.ReLU(),
      torch.nn.Linear(128, dim_item),
    )

    self.optimizer = torch.optim.NAdam(self.parameters(), lr=lr, weight_decay=weight_decay)
    # normalizaton is done in the training step
    self.criterion = torch.nn.MSELoss()

  def train_step(self, x, y):
    self.zero_grad()
    output = self(x)
    loss = self.criterion(output, y)
    loss.backward()
    self.optimizer.step()
    return loss, output

  def fit(self, train_data, batch_size=128, shuffle=True, epochs=20):
    train_dataloader = torch.utils.data.dataloader.DataLoader(train_data, batch_size=batch_size, shuffle=shuffle)
    total_steps = len(train_dataloader)
    print("****************** START TRAINING ******************")
    print("Total steps {:n}".format(total_steps))

    losses = {"mse_train": []}

    for epoch_index in range(1, epochs + 1):
      epoch_start = time.time()
      mse_losses_per_epoch = []
      for step, io_batch in enumerate(train_dataloader, start=1):
        loss, predictions = self.train_step(io_batch, io_batch)
        mse_losses_per_epoch.append(loss.item())
        # log_dict = {
        #   "Epoch": f"{epoch_index} / {epochs}",
        #   "Step": f"{step}/{total_steps}",
        #   "Loss(mse)": round(np.mean(mse_losses_per_epoch), 4),
        #   "Training time": f"{(time.time() - epoch_start):2f}s",
        # }
        # for key, val in log_dict.items():
        #   print(f"{key}: {val}\n")

      losses["mse_train"].append(np.mean(mse_losses_per_epoch))

      train_end = time.time()
      log_dict = {
        "Epoch": f"{epoch_index} / {epochs}",
        "Loss(mse)": round(losses["mse_train"][-1], 4),
        "Training time": f"{(train_end - epoch_start):2f}s",
      }

      for key, val in log_dict.items():
          print(f"{key}: {val}\n")

    print("****************** [END] ******************")
    return losses

  def forward(self, x):
    # weights = torch.vstack([param for param in self.W_list])
    x = x.float()
    # n = weights.shape[0]
    # print(weights.shape)
    # for i in range(n):
    #     weights[i, i] = 0
    # XB = x @ weights
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)

    return decoded

  def predict(self, predict_data, batch_size=128):
    shuffle = False
    predict_dataloader = torch.utils.data.dataloader.DataLoader(predict_data, batch_size=batch_size, shuffle=shuffle)
    prediction = torch.vstack(list(self.predict_generator(predict_dataloader)))

    rm_seen_factor = 1000
    prediction_rm_seen = prediction - rm_seen_factor * predict_data
    return prediction_rm_seen

  def predict_generator(self, predict_dataloader):
    for input_batch in predict_dataloader:
        yield self.forward(input_batch).detach()


In [ ]:
model = Autoencoder(dim_item=user_item_binary_matrix.shape[1])

In [ ]:
model.fit(user_item, epochs=20)

****************** START TRAINING ******************
Total steps 86
Epoch: 1 / 20

Loss(nmse): 51580.8069

Training time: 12.667518s

Epoch: 2 / 20

Loss(nmse): 0.0827

Training time: 12.587092s

Epoch: 3 / 20

Loss(nmse): 0.0054

Training time: 12.645896s

Epoch: 4 / 20

Loss(nmse): 0.0043

Training time: 12.807837s

Epoch: 5 / 20

Loss(nmse): 0.0043

Training time: 14.324224s

Epoch: 6 / 20

Loss(nmse): 0.0043

Training time: 14.246735s

Epoch: 7 / 20

Loss(nmse): 0.0043

Training time: 14.525122s

Epoch: 8 / 20

Loss(nmse): 0.0043

Training time: 14.278309s

Epoch: 9 / 20

Loss(nmse): 0.0043

Training time: 14.305208s

Epoch: 10 / 20

Loss(nmse): 0.0043

Training time: 14.311119s

Epoch: 11 / 20

Loss(nmse): 0.0043

Training time: 14.425828s

Epoch: 12 / 20

Loss(nmse): 0.0043

Training time: 14.501029s

Epoch: 13 / 20

Loss(nmse): 0.0043

Training time: 14.568682s

Epoch: 14 / 20

Loss(nmse): 0.0043

Training time: 14.566627s

Epoch: 15 / 20

Loss(nmse): 0.0043

Training time: 14.6

{'nmse_train': [51580.80689232047,
  0.08272408174762372,
  0.005415027836597589,
  0.004329258814280809,
  0.004287225914425975,
  0.004275779432054002,
  0.004275218288544132,
  0.004281518282368779,
  0.00427433647496929,
  0.004286495262085525,
  0.004272086748874984,
  0.004281376780898765,
  0.004266873784471563,
  0.00427524518589814,
  0.004275581910448192,
  0.004266102884917758,
  0.004278775837900507,
  0.0042847943773796394,
  0.004286462240744122,
  0.0042776130464707695]}

In [ ]:
# for prediction
file_dir = '/content/gdrive/My Drive/msci720_prj/'
file = 'userIds.json'
file_path = os.path.join(file_dir, file)

with open(file_path, "r") as json_file:
    userIds_predict = json.load(json_file)

In [ ]:
user_predict = pd.DataFrame({'userId': userIds_predict})
user_item_binary_matrix_predict = user_item_binary_matrix[user_item_binary_matrix.index.isin(user_predict['userId'])]
user_item_binary_matrix_predict

movieId,1,2,3,4,5,6,7,8,9,10,...,288979,289253,289727,289797,290213,290263,290383,290767,291419,291857
userId,,,,,,,,,,,,,,,,,,,,,
269,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
516,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
776,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
864,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1190,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200955,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
200956,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
200957,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# user_predict = pd.DataFrame({'userId': userIds_predict})
# user_predict = pd.merge(training_set, user_predict, on=["userId"], how="inner") # cannot generate binary matrix using userIds_predict. Doing so will omit movies.
# item_binary_matrix_predict = pd.crosstab(user_predict["userId"], user_predict["movieId"]).clip(upper=1)
user_item_predict = torch.FloatTensor(user_item_binary_matrix_predict.values)
predict = model.predict(user_item_predict)

In [ ]:
user = "userId"
item = "movieId"
user_index = user + "_index"
item_index = item + "_index"

# user_item_binary_matrix = pd.crosstab(training_set[user], training_set[item]).clip(upper=1)
# convert pd dataframe to np array
# data = user_item_binary_matrix.values
# user_item = torch.FloatTensor(data)
# user_item

In [ ]:
user_idMapIndex = training_set[[user]].drop_duplicates()
user_idMapIndex[user_index] = user_idMapIndex.loc[:, user].astype("category").cat.codes

item_idMapIndex = training_set[[item]].drop_duplicates()
item_idMapIndex[item_index] = item_idMapIndex.loc[:, item].astype("category").cat.codes

training_set = pd.merge(training_set, user_idMapIndex, on=[user], how="inner")
training_set = pd.merge(training_set, item_idMapIndex, on=[item], how="inner")

item_idMapIndex_map = {}
for itemId, itemIndex in item_idMapIndex.values:
  item_idMapIndex_map[itemIndex] = itemId

In [ ]:
num_recs = 100

_output_itemId = []
_output_rating = []
for _preds in predict:
  for _index in _preds.topk(num_recs).indices.tolist():
    _output_itemId.append(item_idMapIndex_map[_index])
  for _rating in _preds.topk(num_recs).values.tolist():
    _output_rating.append(_rating)

_output_userId = np.repeat(user_predict[["userId"]].values, num_recs)
Q0 = ["Q0"] * len(_output_userId)
ranking = [i for _ in range(len(user_predict)) for i in range(1, num_recs + 1)]
algo = ["Autoencoder"] * len(_output_userId)
output = pd.DataFrame({'userId': _output_userId, "Q0" : Q0, 'itemId': _output_itemId, 'ranking': ranking, 'rating': _output_rating, 'algo': algo})

In [ ]:
output

,userId,Q0,itemId,ranking,rating,algo
0,269,Q0,318,1,0.043009,Autoencoder
1,269,Q0,296,2,0.038092,Autoencoder
2,269,Q0,2571,3,0.036494,Autoencoder
3,269,Q0,356,4,0.035377,Autoencoder
4,269,Q0,593,5,0.034725,Autoencoder
...,...,...,...,...,...,...
101095,200959,Q0,1721,96,0.010617,Autoencoder
101096,200959,Q0,5445,97,0.010520,Autoencoder
101097,200959,Q0,55820,98,0.010507,Autoencoder
101098,200959,Q0,2716,99,0.010365,Autoencoder


In [ ]:
result = "autoencoder_0.15_1e-3_20.results"
file_dir = "/content/gdrive/My Drive/msci720_prj/result/"
file_save_path = os.path.join(file_dir, result)

output.to_csv(file_save_path, sep=' ', header=None, index=False)